**Tabla de contenido**
- [Librerías](#Lib)
- [Conexión estandar mediante cursor a potsgresql](Conexion-estandar-mediante-cursor-a-potsgresql)
- [Ingestar datos a la base de datos de PostgreSQL con cursor](#Ingestar-datos-a-la-base-de-datos-de-PostgreSQL)
- [Conexión con SQLAlchemy y Pandas](#Conexion-con-SQLAlchemy-y-Pandas)
    - [Creando la conexion a la base de datos](#Creando-la-conexion-a-la-base-de-datos)

# Conexiones a bases de datos con python

En python, el acceso a las bases de datos se encuentra definio a modo estandar en las especificaciones del DB-API y que puedes leer en la PEP 249. Esto significa que independientemente de la base de datos que utilicemos, los métodos y procesos de conexión, lectura y escritura de datos en python, siempren serán los mismos, más allá del conector.

Para desarrollar un proyecto en python en el que se quiera implementar el uso de "Bases de datos SQL", se tienen distintos modulos que pueden ser útiles, por ejemplo "pyodbc", que es uno de ellos.

- **ODBC:** Open Data Base Connectivity es un estandar de acceso a las bases de datos desarrollada por SQL access Group en 1992. El objetivo de ODBC, es hacer posible el acceso a cualquier dato desde cualquier aplicación, sin importar que sistema de gestion de bases de datos almacenen los datos.
- **PYODBC:** Es el modulo de python de código abierto que simplifica el acceso a las bases de datos "ODBC" desde python, implementando el uso de la DB API 2.0 de una forma conveniente para python. Pyodbc tambien es considerado como un controlador de SQL para python.

# Lib

In [1]:
import pandas as pd
import time
import os

pd.options.display.max_columns = None

In [2]:
# Lectura de datos
# https://github.com/YuehHanChen/Marketing_Analytics/blob/main/marketing_data.csv
patt_file = lambda x: os.path.join(os.getcwd(), 'data', x)
df = pd.read_csv(patt_file('marketing_data.csv'), sep=',')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,"$84,835.00",0,0,6/16/14,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,"$57,091.00",0,0,6/15/14,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,"$67,267.00",0,1,5/13/14,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,"$32,474.00",1,1,5/11/14,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,"$21,474.00",1,0,4/8/14,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP


In [3]:
df['Dt_Customer']= pd.to_datetime(df['Dt_Customer'], format='%m/%d/%y')
df.columns=df.columns.str.lower().str.replace(' ', '').str.replace('_','').str.lower() # estandarizar nombres de columnas
regex=r'[$,]'
df['income'] = df['income'].str.replace(regex, '', regex=True).astype(float) # eliminar $ y , de income
df['income'] = df['income'].fillna(0) # reemplazar NaN por 0
df.head()

,id,yearbirth,education,maritalstatus,income,kidhome,teenhome,dtcustomer,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,acceptedcmp3,acceptedcmp4,acceptedcmp5,acceptedcmp1,acceptedcmp2,response,complain,country
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,104,379,111,189,218,1,4,4,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,5,64,7,0,37,1,7,3,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,0,134,11,59,15,2,30,1,3,2,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,32474.0,1,1,2014-05-11,0,10,0,1,0,0,0,1,1,0,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,21474.0,1,0,2014-04-08,0,6,16,24,11,0,34,2,3,1,2,7,1,0,0,0,0,1,0,SP


In [4]:
print(len(df))  # Número de filas
df = pd.concat([df]*2, ignore_index=True)  # Duplicar filas
print(len(df))  # Verificar que se duplicaron las filas

2240
4480


# Conexion estandar mediante cursor a potsgresql

Para poder conectarse a postgreSQL mediante python es necesario tener instalada la siguiente libreria:
- pip install psycopg2-binary

In [ ]:
import psycopg2
import json
# Cargar archivo json con credenciales y configuración de conexión
path = os.getcwd()
path_credenciales =os.path.join(path,'secretos','credenciales_postgresql.json')

with open(path_credenciales, 'r') as file:
    credentials = json.load(file)

# Usar los datos del JSON en la conexión
connection = psycopg2.connect(
    host=credentials['host'],
    port=credentials['port'],
    database=credentials['database'],
    user=credentials['user'],
    password=credentials['password']
)
print("Conexión exitosa")

Conexión exitosa


**Consultemos las tablas que hay en la base de datos**

In [ ]:
cursor = connection.cursor()
sql = """
    SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'public'
    AND table_type = 'BASE TABLE';
"""
connection.rollback() # Limpiar la conexión antes de ejecutar la consulta
cursor.execute(sql)
tables = cursor.fetchall()
print(tables)

[]


Podemos ver que no hay tablas en la base de datos.

# Ingestar datos a la base de datos de PostgreSQL

Para poder ingestar el dataframe, ocuparemos el cursor. Mediante un ciclo for cada fila en el dataframe se convertirá en una tupla y a esa tupla le construiremos una query para ingresar datos en sql.

- Lo primero que tenemos que hacer es saber el tipo de datos que tiene cada columna del dataframe.
- Lo segundo es mapear tipos de Pandas a tipos de SQl (Según la BD)
- Lo tercero es crear la tabla de destino.
- Lo cuarto es ingestar los datos a la tabla.

In [ ]:
# Consultar tipos de datos de las columnas
columnas_tipos = df.dtypes.reset_index()
columnas_tipos.columns = ['columna', 'tipo_dato']
print(columnas_tipos)

                columna       tipo_dato
0                    id           int64
1             yearbirth           int64
2             education          object
3         maritalstatus          object
4                income         float64
5               kidhome           int64
6              teenhome           int64
7            dtcustomer  datetime64[ns]
8               recency           int64
9              mntwines           int64
10            mntfruits           int64
11      mntmeatproducts           int64
12      mntfishproducts           int64
13     mntsweetproducts           int64
14         mntgoldprods           int64
15    numdealspurchases           int64
16      numwebpurchases           int64
17  numcatalogpurchases           int64
18    numstorepurchases           int64
19    numwebvisitsmonth           int64
20         acceptedcmp3           int64
21         acceptedcmp4           int64
22         acceptedcmp5           int64
23         acceptedcmp1           int64


In [28]:
# Consultemos la cantidad de cartacteres mximo de cada columna tipo object
max_length = df.select_dtypes(include=['object']).apply(lambda x: x.str.len().max())
print(max_length)

education        10
maritalstatus     8
country           3
dtype: int64


In [29]:
# Mapeo de tipos de datos de pandas a tipos SQL
tipo_sql = {
    'int64': 'INTEGER',
    'float64': 'DOUBLE PRECISION',
    'object': 'VARCHAR(10)',  # Puedes ajustar el tamaño si lo necesitas
    'datetime64[ns]': 'TIMESTAMP',
    'bool': 'BOOLEAN',
}
columnas_tipos['tipo_sql'] = columnas_tipos['tipo_dato'].astype(str).map(tipo_sql)
print(columnas_tipos[['columna', 'tipo_sql']])

                columna          tipo_sql
0                    id           INTEGER
1             yearbirth           INTEGER
2             education       VARCHAR(10)
3         maritalstatus       VARCHAR(10)
4                income  DOUBLE PRECISION
5               kidhome           INTEGER
6              teenhome           INTEGER
7            dtcustomer         TIMESTAMP
8               recency           INTEGER
9              mntwines           INTEGER
10            mntfruits           INTEGER
11      mntmeatproducts           INTEGER
12      mntfishproducts           INTEGER
13     mntsweetproducts           INTEGER
14         mntgoldprods           INTEGER
15    numdealspurchases           INTEGER
16      numwebpurchases           INTEGER
17  numcatalogpurchases           INTEGER
18    numstorepurchases           INTEGER
19    numwebvisitsmonth           INTEGER
20         acceptedcmp3           INTEGER
21         acceptedcmp4           INTEGER
22         acceptedcmp5           

In [34]:
# Crear la tabla en PostgreSQL
query_create_table = """
CREATE TABLE IF NOT EXISTS marketing (
    id                   INTEGER,
    yearbirth            INTEGER,
    education            VARCHAR(10),
    maritalstatus        VARCHAR(10),
    income               DOUBLE PRECISION,
    kidhome              INTEGER,
    teenhome             INTEGER,
    dtcustomer           TIMESTAMP,
    recency              INTEGER,
    mntwines             INTEGER,
    mntfruits            INTEGER,
    mntmeatproducts      INTEGER,
    mntfishproducts      INTEGER,
    mntsweetproducts     INTEGER,
    mntgoldprods         INTEGER,
    numdealspurchases    INTEGER,
    numwebpurchases      INTEGER,
    numcatalogpurchases  INTEGER,
    numstorepurchases    INTEGER,
    numwebvisitsmonth    INTEGER,
    acceptedcmp3         INTEGER,
    acceptedcmp4         INTEGER,
    acceptedcmp5         INTEGER,
    acceptedcmp1         INTEGER,
    acceptedcmp2         INTEGER,
    response             INTEGER,
    complain             INTEGER,
    country              VARCHAR(10)
);
"""

In [35]:
# Ejecutar la consulta para crear la tabla
cursor.execute(query_create_table)

In [36]:
cursor.execute("""
SELECT EXISTS (
    SELECT FROM information_schema.tables 
    WHERE table_schema = 'public' 
    AND table_name = 'marketing'
);
""")
existe = cursor.fetchone()[0]
print("¿Tabla creada correctamente?:", existe)

¿Tabla creada correctamente?: True


In [38]:
# ingestar los datos a la tabla
name_columns = ", ".join(df.columns)  # sin comillas
placeholders = ", ".join(["%s"] * len(df.columns))

inicio = time.time()

for _, row in df.iterrows():
    sql = f"INSERT INTO marketing ({name_columns}) VALUES ({placeholders})"
    cursor.execute(sql, tuple(row))

connection.commit()

fin = time.time()
print(f"Tiempo de inserción: {fin - inicio:.2f} segundos")


Tiempo de inserción: 0.80 segundos


# Conexion con SQLAlchemy y Pandas

La magia de pandas y SQLAlchemy en acción. Pero primero ¿Qué es un ORM?

ORM o Object Relational Mapper es un software o herramienta que asigna su código a una base de daton sin que ud haga uso de un conector de bases de datos directamente, ya que es abstracts del proceso de conexión de la base de datos. Esto asu vez, nos permite conectarnos a cualquier base de datos relacional sin cambiar demasiado el código.

(La abstracción es el proceso de poner a disposición las características de una herramienta o programa para uso directo sin que nos preocupemos por como funciona)

## Creando la conexion a la base de datos

In [6]:
from sqlalchemy import create_engine
import json
# Ruta al archivo de credenciales
path = os.getcwd()
path_credenciales = os.path.join(path, 'secretos', 'credenciales_postgresql.json')

# Cargar credenciales desde el JSON
with open(path_credenciales, 'r') as file:
    credentials = json.load(file)

# Crear engine con SQLAlchemy para PostgreSQL
engine = create_engine(
    'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'.format(
        user=credentials['user'],
        password=credentials['password'],
        host=credentials['host'],
        port=credentials['port'],
        database=credentials['database']
    )
)
try:
    engine.connect()
    print("Conexión exitosa con SQLAlchemy")
except Exception as e:
    print("Error al conectar con SQLAlchemy:", e)   

Conexión exitosa con SQLAlchemy


Con SQLAlchemy y pandas podemos hacer lo siguiente:

- Crear la tabla en la base de datos si esta no existe.
- Utilizar un chucksize para ingestar de golpe un rango de filas
- Con el método "multi" podemos ejecutar varias injestas a la vez.
- Con if_exist te da opciones para agregar o borrar los datos si la tabla ya está creada y tiene datos.

Cuando usas **pandas.DataFrame.to_sql()** por defecto, no es obligatorio especificar los tipos de datos. pandas junto con SQLAlchemy hace una inferencia automática de los tipos de datos basándose en el contenido del DataFrame.

In [7]:
# Nombre de la tabla que quieres crear en la base de datos
nombre_tabla = 'marketing'

# Cargar el DataFrame a la base de datos (creará la tabla si no existe)
inicio = time.time()
try:
    df.to_sql(nombre_tabla, con =engine, if_exists='replace', index=False)
    print(f"Tabla '{nombre_tabla}' creada y datos cargados exitosamente.")
except Exception as e:
    print("Error al crear la tabla o cargar los datos:", e)
fin = time.time()
print(f"Tiempo de inserción con SQLAlchemy: {fin - inicio:.2f} segundos")

Tabla 'marketing' creada y datos cargados exitosamente.
Tiempo de inserción con SQLAlchemy: 0.48 segundos
